In [ ]:
################################################################################
# INSTALL CONDA ON GOOGLE COLAB
################################################################################
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2022-08-03 17:15:27--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   102MB/s    in 0.8s    

2022-08-03 17:15:28 (102 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b64

In [ ]:
! conda create -n rapids-22.06 -c rapidsai -c nvidia -c conda-forge  \
    cuml=22.06 python=3.9 cudatoolkit=11.5

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 

In [ ]:
# ! conda create -n rapids-22.04 -c rapidsai -c nvidia -c conda-forge  \
    # cuml=22.04 python=3.9 cudatoolkit=11.5

In [ ]:
# activate your conda environment
%%bash
source activate rapids-22.06
pip install bertopic
pip install -U kaleido
pip install gensim

python
import sys
print("Python version")
print (sys.version)
from cuml.cluster import HDBSCAN
from cuml.manifold import UMAP
from bertopic import BERTopic
umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0)
hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True)

import numpy as np
import pandas as pd

#load data
data = pd.read_csv('data_all_072022.csv', nrows =10)
data.drop('Unnamed: 0', axis=1, inplace=True)

#text preprocessing
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')
stop_words_hinglish = pd.read_csv('stop_hinglish.txt')
stop_words_hindi = pd.read_csv('stop_hindi.txt')


data['translated_text_all'] = data['translated_text_all'].str.replace("#", " ")

def clean_text(x):
    x = str(x)
    x = x.lower()
    x = re.sub(r'#[A-Za-z0-9]*', ' ', x)
    x = re.sub(r'@[A-Za-z0-9]+', ' ', x)
    
    tokens = word_tokenize(x)
    x = ' '.join([w for w in tokens if not w.lower() in stop_words])
    x = ' '.join([w for w in tokens if not w.lower() in stop_words_hinglish])
    x = ' '.join([w for w in tokens if not w.lower() in stop_words_hindi])
    x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
    x = re.sub(r'\d+', ' ', x)
    x = re.sub(r'\n+', ' ', x)
    return x


data['clean_text'] = data.translated_text_all.apply(clean_text)
data['tokenized_sents'] = data.apply(lambda row: nltk.word_tokenize(row['clean_text']), axis=1)

word_list = list(chain.from_iterable(data.tokenized_sents.values))
print(word_list)
words = ' '.join(word_list)

topicmodel1 = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model, calculate_probabilities=False, nr_topics=10)
topics,probs = topic_model1.fit_transform(word_list)

topic_model1.visualize_barchart()

topic_model1.visualize_heatmap(top_n_topics = 20)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Python version
3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:56:21) 
[GCC 10.3.0]


Traceback (most recent call last):
  File "<stdin>", line 4, in <module>
  File "/usr/local/envs/rapids-22.04/lib/python3.9/site-packages/cuml/__init__.py", line 17, in <module>
    from cuml.common.base import Base
  File "/usr/local/envs/rapids-22.04/lib/python3.9/site-packages/cuml/common/__init__.py", line 17, in <module>
    from cuml.common.array import CumlArray
  File "/usr/local/envs/rapids-22.04/lib/python3.9/site-packages/cuml/common/array.py", line 25, in <module>
    from cudf import DataFrame
  File "/usr/local/envs/rapids-22.04/lib/python3.9/site-packages/cudf/__init__.py", line 5, in <module>
    validate_setup()
  File "/usr/local/envs/rapids-22.04/lib/python3.9/site-packages/cudf/utils/gpu_utils.py", line 20, in validate_setup
    from rmm._cuda.gpu import (
  File "/usr/local/envs/rapids-22.04/lib/python3.9/site-packages/rmm/__init__.py", line 16, in <module>
    from rmm import mr
  File "/usr/local/envs/rapids-22.04/lib/python3.9/site-packages/rmm/mr.py", line 14, 